In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/08 11:33:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/2/athlete_events.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("athlete_events.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show DataFrame
df.show()

+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
| ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|       City|               Sport|               Event|Medal|
+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
|  1|           A Dijiang|  M| 24|   180|    80|         China|CHN|1992 Summer|1992|Summer|  Barcelona|          Basketball|Basketball Men's ...|   NA|
|  2|            A Lamusi|  M| 23|   170|    60|         China|CHN|2012 Summer|2012|Summer|     London|                Judo|Judo Men's Extra-...|   NA|
|  3| Gunnar Nielsen Aaby|  M| 24|    NA|    NA|       Denmark|DEN|1920 Summer|1920|Summer|  Antwerpen|            Football|Football Men's Fo...|   NA|
|  4|Edgar Lindenau Aabye|  M| 34|    NA|    NA|Denmark/Sweden|DEN|1900 Summer|1900|Summ

In [4]:
#Create a temporary view.
df.createOrReplaceTempView('games')

In [5]:
# Get the numover of distinct sports played in 2016
spark.sql("SELECT count(distinct sport) FROM games WHERE year=2016").show()

+---------------------+
|count(DISTINCT sport)|
+---------------------+
|                   34|
+---------------------+



In [6]:
# Get the number of distinct sports played in 2012
spark.sql("SELECT count(distinct sport) FROM games WHERE year=2012").show()

+---------------------+
|count(DISTINCT sport)|
+---------------------+
|                   32|
+---------------------+



In [7]:
# Write a subquery to get the distinct sports played in 2016 but not in 2012.
spark.sql("""
    SELECT DISTINCT sport 
    FROM games 
    WHERE year=2016 AND sport NOT IN 
      (SELECT DISTINCT sport 
      FROM games 
      WHERE year=2012)
    """).show()

+------------+
|       sport|
+------------+
|        Golf|
|Rugby Sevens|
+------------+



In [8]:
# Another option could be using an EXCEPT
spark.sql("""
    SELECT DISTINCT sport 
    FROM games 
    WHERE year=2016 
    EXCEPT
        SELECT 
      distinct
      sport 
    FROM games 
    WHERE year=2012
    """).show()

+------------+
|       sport|
+------------+
|        Golf|
|Rugby Sevens|
+------------+



Question 2: Which sports have been played since the first summer Olympics?

In [9]:
# Get the year of the earliest Olympics in our data
spark.sql("""SELECT min(year) FROM games""").show()

+---------+
|min(year)|
+---------+
|     1896|
+---------+



In [10]:
# Get the sports that were played in the earliest Olympic games in our data
spark.sql("""
      SELECT 
      DISTINCT sport 
      FROM games 
      WHERE year=1896""").show()

+-------------+
|        sport|
+-------------+
|   Gymnastics|
|       Tennis|
|     Swimming|
|    Athletics|
|     Shooting|
|      Cycling|
|      Fencing|
|    Wrestling|
|Weightlifting|
+-------------+



In [11]:
# Determine which sports were in both the 1896 Olympics and the 2016 Olympics
spark.sql("""
  SELECT 
    DISTINCT 
      a.sport 
  FROM games a 
    JOIN 
        (SELECT DISTINCT sport FROM games WHERE year=1896) b 
    ON 
      a.sport=b.sport 
  WHERE a.year=2016""").show()

+-------------+
|        sport|
+-------------+
|   Gymnastics|
|       Tennis|
|     Swimming|
|    Athletics|
|     Shooting|
|    Wrestling|
|Weightlifting|
|      Cycling|
|      Fencing|
+-------------+



Question 3: Who are the top 5 athletes that won most gold, silver and bronze medals?

In [12]:
# Who are the top 5 Gold medal winners in the Olympics 
spark.sql("""
SELECT 
  Name,
  medal, 
  count(*) as medalCount 
FROM games 
WHERE medal ='Gold' GROUP BY Name, medal 
ORDER BY medalCount desc
LIMIT 5 
""").show(truncate=False)


+----------------------------------+-----+----------+
|Name                              |medal|medalCount|
+----------------------------------+-----+----------+
|Michael Fred Phelps, II           |Gold |23        |
|Raymond Clarence "Ray" Ewry       |Gold |10        |
|Larysa Semenivna Latynina (Diriy-)|Gold |9         |
|Frederick Carlton "Carl" Lewis    |Gold |9         |
|Mark Andrew Spitz                 |Gold |9         |
+----------------------------------+-----+----------+



In [13]:
# Who are the top 5 Silver medal winners
spark.sql("""
SELECT 
  Name,
  medal, 
  count(*) as medalCount 
FROM games 
WHERE medal ='Silver' GROUP BY Name, medal 
ORDER BY medalCount desc
LIMIT 5 
""").show(truncate=False)


+-------------------------------+------+----------+
|Name                           |medal |medalCount|
+-------------------------------+------+----------+
|Aleksandr Nikolayevich Dityatin|Silver|6         |
|Shirley Frances Babashoff      |Silver|6         |
|Aleksandr Vladimirovich Popov  |Silver|6         |
|Mikhail Yakovlevich Voronin    |Silver|6         |
|Yang Yang                      |Silver|6         |
+-------------------------------+------+----------+



In [14]:
# Who are the top 5 Bronze medal winners
spark.sql("""
SELECT 
  Name,
  medal, 
  count(*) as medalCount 
FROM games 
WHERE medal ='Bronze' GROUP BY Name, medal 
ORDER BY medalCount desc
LIMIT 5 
""").show(truncate=False)

+------------------------+------+----------+
|Name                    |medal |medalCount|
+------------------------+------+----------+
|Heikki Ilmari Savolainen|Bronze|6         |
|Harri Tapani Kirvesniemi|Bronze|6         |
|Merlene Joyce Ottey-Page|Bronze|6         |
|Aleksey Yuryevich Nemov |Bronze|6         |
|Franziska van Almsick   |Bronze|6         |
+------------------------+------+----------+



Output the gold medal winners to csv

Hint: since we have aggregated our data it is safe to go back to pandas

In [15]:
import pandas as pd

pandas_df=spark.sql("""
SELECT 
  Name,
  medal, 
  count(*) as medalCount 
FROM games 
WHERE medal ='Gold' GROUP BY Name, medal 
ORDER BY medalCount desc
LIMIT 5 
""").toPandas()

pandas_df.to_csv('top5Gold.csv')